In [1]:
import csv
import collections
import statistics

In [76]:
with open('/global/scratch/users/pierrj/eccDNA/magnaporthe_pureculture/rawdata/illumina/G3_1A_tests/ecc_confidence') as coverage:
    coverage_reader = csv.reader(coverage, delimiter = '\t')
    coverage_list = [[] for i in range(56)]
    for row in coverage_reader:
        coverage_list[int(row[0])].append([int(row[1]), int(row[2]), str(row[3]), str(row[4]), str(row[5])])

In [ ]:
## name eccs (Chrom[num]ecc[num])
## maybe do this after actually

In [77]:
def uniq_sort(start_list):
    uniq_list = [list(x) for x in set(tuple(x) for x in start_list)]
    end_list = sorted(uniq_list,key=lambda x: x[0])
    return end_list

In [78]:
sortuniqcoverage_list_chrom0 = uniq_sort(coverage_list[0])

In [116]:
print(sortuniqcoverage_list_chrom0)

[[2999, 3499], [3000, 3500], [3001, 3501], [3051, 3551], [3052, 3552], [3053, 3553]]


In [79]:
ecc_withvariants = {}
for i in range(len(sortuniqcoverage_list_chrom0)):
    ecc = sortuniqcoverage_list_chrom0[i]
    variants = []
## pretty major issue is that this does a rolling thing so some eccs probably appear twice
    for k in range(len(sortuniqcoverage_list_chrom0)):
        start_coordinate1 = sortuniqcoverage_list_chrom0[k][0] - 50
        start_coordinate2 = sortuniqcoverage_list_chrom0[k][0] + 50
        end_coordinate1 = sortuniqcoverage_list_chrom0[k][1] - 50
        end_coordinate2 = sortuniqcoverage_list_chrom0[k][1] + 50
        if start_coordinate1 <= ecc[0] <= start_coordinate2 and end_coordinate1 <= ecc[1] <= end_coordinate2:
            variants.append(sortuniqcoverage_list_chrom0[k])
    if tuple([ecc]) != tuple(variants): ## if there isn't just itself in the list
        ecc_withvariants[tuple(ecc)] = variants
variants_grouped = collections.defaultdict(list)
for key,val in ecc_withvariants.items(): ## group eccs with same variants
    variants_grouped[tuple(tuple(x) for x in val)].append(key)

In [61]:
def getcoord(coord_list): ## get ideal representative coordinates based off medians
    median = statistics.median(coord_list)
    distancetoend = abs(median - max(coord_list))
    distancetostart = abs(median - min(coord_list))
    greaterlist = [i for i in coord_list if i >= median]
    smallerlist = [i for i in coord_list if i <= median]
    #if median is closer to end, get value above median. if median is closer to start, get value below median
    #if equal get both values, one as alt
    if distancetoend > distancetostart:
        coord = min(smallerlist, key=lambda x:abs(x-median))
        coord_alt = 'N/A'
    elif distancetoend == distancetostart:
        coord = min(greaterlist, key=lambda x:abs(x-median))
        coord_alt = min(smallerlist, key=lambda x:abs(x-median))
    elif distancetoend < distancetostart:
        coord = min(greaterlist, key=lambda x:abs(x-median))
        coord_alt = 'N/A'
    else:
        raise ValueError("getcoord error")
    return coord, coord_alt

In [62]:
def test_coords(target_coord, other_coord, target_start_or_end): ## get closest matching coordinate based off on set/forced coordinate
        ## check if start or end coordinate is the set one
        if target_start_or_end == 'start':
            known = 0
            unknown = 1
        if target_start_or_end == 'end':
            known = 1
            unknown = 0
        ## get all potential options matching set coordinate
        unknown_options = []
        for i in val:
            if i[known] == target_coord:
                unknown_options.append(i[unknown])
        ## test which one is the closest to the ideal median
        unknown_coord = min(unknown_options, key=lambda x:abs(x-other_coord))
        return unknown_coord

def reconcile_coords(startcoord, endcoord): ## test which coordinate pair is closest to ideal
        #test with start coordinate forced
        startcoord_forced = test_coords(startcoord, endcoord, 'start')
        #test with end coordinate forced
        endcoord_forced = test_coords(endcoord, startcoord, 'end')
        #test how close each non-ideal corresponding coordinates are to the ideal
        fit_startcoord_forced = abs(startcoord_forced - endcoord)
        fit_endcoord_forced = abs(endcoord_forced - startcoord)
        if fit_startcoord_forced <= fit_endcoord_forced:
            return startcoord, startcoord_forced
        else:
            return endcoord_forced, endcoord

In [80]:
representative_variants = {}
for key, val in variants_grouped.items():
    startlist = []
    endlist = []
    for i in val:
        startlist.append(i[0])
        endlist.append(i[1])
    start_coord, start_coord_alt = getcoord(startlist)
    end_coord, end_coord_alt = getcoord(endlist)
    middle_ecc = 0
    ## check to see if a single ecc has ideal coordinates for both end and start
    for i in val:
        if i[0] == start_coord and i[1] == end_coord:
            middle_ecc = i
        if i[0] == start_coord and i[1] == end_coord_alt:
            middle_ecc = i
        if i[0] == start_coord_alt and i[1] == end_coord:
            middle_ecc = i
        if i[0] == start_coord and i[1] == start_coord_alt:
            middle_ecc = i
    ## if not then reconcile coordinates
    if middle_ecc == 0:
        start_coord_estimate, end_coord_estimate = reconcile_coords(start_coord, end_coord)
        for i in val:
            if i[0] == start_coord_estimate and i[1] == end_coord_estimate:
                middle_ecc = i
    if middle_ecc == 0:
        raise ValueError("No ecc merged ecc called")
    else:
        representative_variants[middle_ecc] = val

In [81]:
print(len(sortuniqcoverage_list_chrom0))
premerge_uniq_sortuniqcoverage_list_chrom0 = [list(x) for x in set(tuple(x) for x in sortuniqcoverage_list_chrom0)]
print(len(premerge_uniq_sortuniqcoverage_list_chrom0))
for i in range(len(sortuniqcoverage_list_chrom0)):
    ecc = sortuniqcoverage_list_chrom0[i]
    for key, val in representative_variants.items():
        if tuple(ecc) in val:
            sortuniqcoverage_list_chrom0[i] = list(key)
            #sortuniqcoverage_list_chrom0[i].append('yes')
        #else:
            #sortuniqcoverage_list_chrom0[i].append('no')
print(len(sortuniqcoverage_list_chrom0))
print(len(ecc_withvariants))
print(len(representative_variants))
uniq_sortuniqcoverage_list_chrom0 = [list(x) for x in set(tuple(x) for x in sortuniqcoverage_list_chrom0)]
print(len(uniq_sortuniqcoverage_list_chrom0))

#print(uniq_sortuniqcoverage_list_chrom0)
sorted(uniq_sortuniqcoverage_list_chrom0,key=lambda x: x[0]) == uniq_sort(sortuniqcoverage_list_chrom0)
#print(sorted(uniq_sortuniqcoverage_list_chrom0,key=lambda x: x[0]))
#print(uniq_sort(sortuniqcoverage_list_chrom0))

11476
11476
11476
8378
2483
5581


True

In [74]:
print(len(uniq_sort(sortuniqcoverage_list_chrom0)))

11473


In [9]:
print(representative_variants)

{(3269, 3771, 'conf', 'coverage_too_low', '39.61;54.29;29.86'): [(3269, 3796, 'conf', 'coverage_too_low', '38.35;54.12;29.12'), (3269, 3809, 'conf', 'coverage_too_low', '37.7;53.82;28.9'), (3269, 3767, 'conf', 'coverage_too_low', '39.82;54.31;29.96'), (3269, 3771, 'conf', 'coverage_too_low', '39.61;54.29;29.86')], (4158, 6063, 'conf', 'coverage_too_low', '32.96;94.78;109.82'): [(4158, 6073, 'conf', 'coverage_too_low', '32.83;95.17;109.15'), (4158, 6063, 'conf', 'coverage_too_low', '32.96;94.78;109.82')], (4929, 5720, 'conf', 'coverage_too_low', '35.05;137.71;132.46'): [(4929, 5737, 'conf', 'coverage_too_low', '34.74;137.28;133.04'), (4929, 5720, 'conf', 'coverage_too_low', '35.05;137.71;132.46')], (5068, 8394, 'hconf', 'none', '31.52;113.92;24.0'): [(5068, 8394, 'hconf', 'none', '31.52;113.92;24.0'), (5068, 8408, 'hconf', 'none', '31.41;113.69;23.87'), (5068, 8386, 'hconf', 'none', '31.58;114.05;24.05')], (12244, 13704, 'conf', 'coverage_too_low', '18.85;126.55;64.09'): [(12244, 13712,

In [24]:
print(sortuniqcoverage_list_chrom0)

[[387, 774, 'conf', 'too_close_to_start', 'N/A;182.11;180.68'], [2881, 3398, 'conf', 'coverage_too_low', '11.68;54.28;42.91'], [3269, 3771, 'conf', 'coverage_too_low', '39.61;54.29;29.86'], [3269, 3771, 'conf', 'coverage_too_low', '39.61;54.29;29.86'], [3269, 3771, 'conf', 'coverage_too_low', '39.61;54.29;29.86'], [3269, 3771, 'conf', 'coverage_too_low', '39.61;54.29;29.86'], [4158, 6063, 'conf', 'coverage_too_low', '32.96;94.78;109.82'], [4158, 6063, 'conf', 'coverage_too_low', '32.96;94.78;109.82'], [4856, 6003, 'conf', 'coverage_too_low', '32.68;128.51;116.77'], [4929, 5720, 'conf', 'coverage_too_low', '35.05;137.71;132.46'], [4929, 5720, 'conf', 'coverage_too_low', '35.05;137.71;132.46'], [5068, 8394, 'hconf', 'none', '31.52;113.92;24.0'], [5068, 8394, 'hconf', 'none', '31.52;113.92;24.0'], [5068, 8394, 'hconf', 'none', '31.52;113.92;24.0'], [5416, 6411, 'conf', 'coverage_too_low', '89.89;128.62;112.8'], [5808, 8095, 'conf', 'coverage_too_low', '73.15;110.54;33.48'], [5903, 7994, '

In [15]:
singlets_list = []
for key, val in representative_variants.items():
    if len(val) == 1:
        print(val)
        print(key)
        singlets_list.append(val)

[(13268, 13860, 'conf', 'coverage_too_low', '152.47;148.94;65.17')]
(13268, 13860, 'conf', 'coverage_too_low', '152.47;148.94;65.17')
[(13268, 13901, 'conf', 'coverage_too_low', '148.02;148.44;57.58')]
(13268, 13901, 'conf', 'coverage_too_low', '148.02;148.44;57.58')
[(126712, 127502, 'conf', 'coverage_too_low', '25.54;96.78;51.84')]
(126712, 127502, 'conf', 'coverage_too_low', '25.54;96.78;51.84')
[(126712, 127555, 'conf', 'coverage_too_low', '25.22;96.51;50.11')]
(126712, 127555, 'conf', 'coverage_too_low', '25.22;96.51;50.11')
[(153432, 154768, 'conf', 'coverage_too_low', '100.73;135.83;34.15')]
(153432, 154768, 'conf', 'coverage_too_low', '100.73;135.83;34.15')
[(153432, 154819, 'conf', 'coverage_too_low', '101.27;137.09;29.77')]
(153432, 154819, 'conf', 'coverage_too_low', '101.27;137.09;29.77')
[(368207, 369218, 'hconf', 'none', '146.35;30092.19;9768.47')]
(368207, 369218, 'hconf', 'none', '146.35;30092.19;9768.47')
[(368212, 369366, 'hconf', 'none', '141.52;28825.57;7364.23')]
(

In [16]:
print(len(sortuniqcoverage_list_chrom0))
print(len(ecc_withvariants))
print(len(representative_variants))
print(len(singlets_list))
print(len(premerge))
print(len(postmerge))

12930
8378
2483
106
12930
2483


In [11]:
premerge = []
for i in range(len(sortuniqcoverage_list_chrom0)):
    to_add = []
    to_add.append('MQOP01000001.1')
    to_add.append(sortuniqcoverage_list_chrom0[i][0])
    to_add.append(sortuniqcoverage_list_chrom0[i][1])
    premerge.append(to_add)

postmerge = []
for key in representative_variants.keys():
    to_add = []
    to_add.append('MQOP01000001.1')
    to_add.append(key[0])
    to_add.append(key[1])
    postmerge.append(to_add)

with open('/global/home/users/pierrj/testfiles/test_premerge', 'w', newline = '') as premerge_file:
    w = csv.writer(premerge_file, delimiter = '\t')
    w.writerows(premerge)

with open('/global/home/users/pierrj/testfiles/test_postmerge', 'w', newline = '') as postmerge_file:
    w = csv.writer(postmerge_file, delimiter = '\t')
    w.writerows(postmerge)

In [61]:
def check_duplicates(dictionary):
    check_list = []
    for key, value in dictionary.items(): 
        check_list.append(value)
    flat_list = [item for sublist in check_list for item in sublist]
    if len(flat_list) == len(set(flat_list)):
        return True
    else:
        raise ValueError('duplicate eccs found in ecc_merge')

## this doesn't work because values are whole lists in stead of each item

## add all values in a dictionary to a list of lists
## turn list of lists in list
## then check for uniques

In [204]:
print(check_duplicates(representative_variants))

True


In [ ]:
## loop through items in the list
## replace them with the key if they are
## nvm that's super duper slow
## how about just deleting them from the list when you pick them out
## then add the keys back into the list

In [ ]:
### count how many eccs are within 100 bp of them on either side
### pick the ones with the highest counts somehow?
## something about the length?
## make a dictonary where the key is the ecc and the values are the lists that ecc is next to, including itself
## then delete keys with duplicate values
## then keep key that is in the middle